In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import aifc
import os
import tensorflow as tf
import random
import pandas as pd
import load_and_generate
from load_and_generate import maybe_extract, maybe_pickle

try:
   import cPickle as pickle
except:
   import pickle

In [2]:
def get_file_sequence(file_name, header):
    return file_name.split('.')[0].split(header)[1]

In [3]:
from os import listdir
from os.path import isfile, join

data_loc = '../whale-inputs/data' 
train_folder = 'train'
file_path = os.path.join(data_loc,train_folder) #change to test if desired

file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]

In [1]:
def merge_datasets(pickle_files, train_size, valid_size=0):
  num_classes = len(pickle_files)
  valid_dataset, valid_labels = make_arrays(valid_size, image_size)
  train_dataset, train_labels = make_arrays(train_size, image_size)
  vsize_per_class = valid_size // num_classes
  tsize_per_class = train_size // num_classes
    
  start_v, start_t = 0, 0
  end_v, end_t = vsize_per_class, tsize_per_class
  end_l = vsize_per_class+tsize_per_class
  for label, pickle_file in enumerate(pickle_files):       
    try:
      with open(pickle_file, 'rb') as f:
        letter_set = pickle.load(f)
        # let's shuffle the letters to have random validation and training set
        np.random.shuffle(letter_set)
        if valid_dataset is not None:
          valid_letter = letter_set[:vsize_per_class, :, :]
          valid_dataset[start_v:end_v, :, :] = valid_letter
          valid_labels[start_v:end_v] = label
          start_v += vsize_per_class
          end_v += vsize_per_class
                    
        train_letter = letter_set[vsize_per_class:end_l, :, :]
        train_dataset[start_t:end_t, :, :] = train_letter
        train_labels[start_t:end_t] = label
        start_t += tsize_per_class
        end_t += tsize_per_class
    except Exception as e:
      print('Unable to process data from', pickle_file, ':', e)
      raise
    
  return valid_dataset, valid_labels, train_dataset, train_labels
            

# Train dataset

In [5]:
image_width = 32
image_height = 32

train_folders = maybe_extract(os.path.abspath( '../whale-inputs/data/train/'))
train_datasets = maybe_pickle(train_folders, 7000, False, image_height, image_width )

['/home/mabelvj/Git/whale-inputs/data/train/no_whales', '/home/mabelvj/Git/whale-inputs/data/train/whales']
/home/mabelvj/Git/whale-inputs/data/train/no_whales.pickle already present - Skipping pickling.
/home/mabelvj/Git/whale-inputs/data/train/whales.pickle already present - Skipping pickling.


In [10]:
# Extraction of whole dataset
v_, v_, train_dataset, train_labels = merge_datasets(train_datasets, 7027, 0)

NameError: global name 'make_arrays' is not defined

In [9]:
train_datasets[1]

'/home/mabelvj/Git/whale-inputs/data/train/whales.pickle'

In [4]:

from load_and_generate import open_pickle

data_loc = '../whale-inputs/data' 
train_folder = join(data_loc, 'train')
test_folder = join(data_loc, 'test')

train_datasets = open_pickle([train_folder])
#test_datasets = open_pickle([test_folder])



KeyboardInterrupt: 

In [5]:
x= random.sample(train_datasets,1)
plt.imshow(train_datasets[1])
reload(load_and_generate)
from load_and_generate import *
load_and_generate.create_label_folders('../whale-inputs/data/', 'train','train.csv')

NameError: name 'train_datasets' is not defined

# test

In [20]:
import sklearn
from sklearn.cross_validation import train_test_split#, StratifiedShuffleSplit
from sklearn.model_selection import StratifiedShuffleSplit
from load_and_generate import *



#Adjusting to freqs 250Hz : height 33
image_height = 33
image_width = 23

#Rescale width
image_height = 32
image_width = 32

image_size = image_height*image_width
#create_label_folders('../whale-inputs/data/', 'test', 'test.csv')

test_folders = maybe_extract(os.path.abspath( '../whale-inputs/data/test/'))
test_datasets = maybe_pickle(test_folders, 10000, False, image_height, image_width )




def make_arrays(nb_rows, image_height, image_width):
  if nb_rows:
    dataset = np.ndarray((nb_rows,  image_height, image_width), dtype=np.float32) #change to dif
    labels = np.ndarray(nb_rows, dtype=np.int32)
  else:
    dataset, labels = None, None
  return dataset, labels

def load_datasets(pickle_files, train_size):
  num_classes = len(pickle_files)
  train_dataset, train_labels = [],[]

  for label, pickle_file in enumerate(pickle_files):       
    try:
      with open(pickle_file, 'rb') as f:
        letter_set = pickle.load(f)
        if train_dataset is not None:
            train_dataset = np.concatenate((train_dataset, letter_set), axis=0)
            train_labels = np.concatenate((train_labels,np.ones(letter_set.shape[0]* label)),axis=0)
        else:
            train_dataset = letter_
    except Exception as e:
      print('Unable to process data from', pickle_file, ':', e)
      raise 
  print train_dataset.shape
      
  return train_dataset, train_labels
            

['/home/mabelvj/Git/whale-inputs/data/test/no_whales', '/home/mabelvj/Git/whale-inputs/data/test/whales']
/home/mabelvj/Git/whale-inputs/data/test/no_whales.pickle already present - Skipping pickling.
/home/mabelvj/Git/whale-inputs/data/test/whales.pickle already present - Skipping pickling.


In [21]:
test_dataset, test_labels = load_datasets(test_datasets, 54503)

('Unable to process data from', '/home/mabelvj/Git/whale-inputs/data/test/no_whales.pickle', ':', ValueError('all the input arrays must have same number of dimensions',))


ValueError: all the input arrays must have same number of dimensions

In [13]:
np.concatenate((test_dataset[0],test_dataset[1]), axis= 0)

array([[[ -175.,  -165.,  -154., ...,  -191.,  -138.,  -126.],
        [ -175.,  -165.,  -154., ...,  -191.,  -138.,  -126.],
        [ -175.,  -165.,  -154., ...,  -191.,  -138.,  -126.],
        ..., 
        [  123.,   425.,   652., ...,  -191.,   854.,  1169.],
        [  662.,   861.,   735., ...,     6.,   171.,   249.],
        [  123.,    52.,    12., ...,   679.,   419.,  -126.]],

       [[ 1206.,  1193.,  1252., ...,  1179.,  1194.,  1182.],
        [  575.,   542.,   465., ...,   593.,   572.,   561.],
        [ -107.,  -124.,  -129., ...,  -112.,  -121.,  -113.],
        ..., 
        [  191.,    74.,   -83., ...,   -82.,    61.,   189.],
        [ -107.,  -107.,    32., ...,   -14.,     5.,   -20.],
        [  -81.,   -81.,     9., ...,   -14.,   -25.,   -43.]],

       [[ -192.,  -251.,  -222., ...,  -183.,  -202.,  -159.],
        [ -192.,  -251.,  -222., ...,  -183.,  -202.,  -159.],
        [ -192.,  -251.,  -222., ...,  -183.,  -202.,  -159.],
        ..., 
        [